In [1]:

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 설정 (Windows & Mac 대응)
plt.rcParams["font.family"] = "Malgun Gothic"  # Windows (맑은 고딕)
plt.rcParams["axes.unicode_minus"] = False  # 마이너스 기호 깨짐 방지


In [4]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import ast

# ✅ SQLite 데이터베이스 연결
db_path = "asia (2-2).db"  # DB 파일 경로
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ 이미지 저장 폴더 생성
output_dir = "Language_charts"
os.makedirs(output_dir, exist_ok=True)

try:
    # ✅ SQL 쿼리 실행 (DB에서 데이터 가져오기)
    sql_query = """
    WITH total_count AS (
        SELECT category AS duty_category, COUNT(*) AS total_jobs
        FROM (
            SELECT 'PM' AS category, language FROM processing WHERE duty LIKE '%PM%'
            UNION ALL
            SELECT '데이터 직무' AS category, language FROM processing WHERE duty LIKE '%데이터 직무%'
            UNION ALL
            SELECT '백엔드' AS category, language FROM processing WHERE duty LIKE '%백엔드%'
            UNION ALL
            SELECT '인프라 엔지니어' AS category, language FROM processing WHERE duty LIKE '%인프라 엔지니어%'
            UNION ALL
            SELECT '앱 개발자' AS category, language FROM processing WHERE duty LIKE '%앱 개발자%'
            UNION ALL
            SELECT '게임' AS category, language FROM processing WHERE duty LIKE '%게임%'
            UNION ALL
            SELECT 'AI' AS category, language FROM processing WHERE duty LIKE '%AI%'
            UNION ALL
            SELECT '임베디드' AS category, language FROM processing WHERE duty LIKE '%임베디드%'
            UNION ALL
            SELECT '프론트 엔드' AS category, language FROM processing WHERE duty LIKE '%프론트 엔드%'
            UNION ALL
            SELECT 'QA' AS category, language FROM processing WHERE duty LIKE '%QA%'
            UNION ALL
            SELECT '데이터 분석' AS category, language FROM processing WHERE duty LIKE '%데이터 분석%'
            UNION ALL
            SELECT 'VR' AS category, language FROM processing WHERE duty LIKE '%VR%'
            UNION ALL
            SELECT '시스템' AS category, language FROM processing WHERE duty LIKE '%시스템%'
            UNION ALL
            SELECT '블록체인' AS category, language FROM processing WHERE duty LIKE '%블록체인%'
            UNION ALL
            SELECT 'ERP' AS category, language FROM processing WHERE duty LIKE '%ERP%'
            UNION ALL
            SELECT '언어별 개발자' AS category, language FROM processing WHERE duty LIKE '%언어별 개발자%'
        ) AS filtered_data
        GROUP BY category
    )

    SELECT 
        f.category AS 직무,
        CASE 
            WHEN f.language = 1 THEN '영어'
            WHEN f.language = 2 THEN '일본어'
            WHEN f.language = 3 THEN '중국어'
            ELSE '무관/없음'
        END AS 어학조건,
        COUNT(*) AS 채용_공고_개수,
        t.total_jobs AS 총_공고_개수,
        ROUND(100.0 * COUNT(*) / t.total_jobs, 2) || '%' AS 비율
    FROM (
        SELECT 'PM' AS category, language FROM processing WHERE duty LIKE '%PM%'
        UNION ALL
        SELECT '데이터 직무' AS category, language FROM processing WHERE duty LIKE '%데이터 직무%'
        UNION ALL
        SELECT '백엔드' AS category, language FROM processing WHERE duty LIKE '%백엔드%'
        UNION ALL
        SELECT '인프라 엔지니어' AS category, language FROM processing WHERE duty LIKE '%인프라 엔지니어%'
        UNION ALL
        SELECT '앱 개발자' AS category, language FROM processing WHERE duty LIKE '%앱 개발자%'
        UNION ALL
        SELECT '게임' AS category, language FROM processing WHERE duty LIKE '%게임%'
        UNION ALL
        SELECT 'AI' AS category, language FROM processing WHERE duty LIKE '%AI%'
        UNION ALL
        SELECT '임베디드' AS category, language FROM processing WHERE duty LIKE '%임베디드%'
        UNION ALL
        SELECT '프론트 엔드' AS category, language FROM processing WHERE duty LIKE '%프론트 엔드%'
        UNION ALL
        SELECT 'QA' AS category, language FROM processing WHERE duty LIKE '%QA%'
        UNION ALL
        SELECT '데이터 분석' AS category, language FROM processing WHERE duty LIKE '%데이터 분석%'
        UNION ALL
        SELECT 'VR' AS category, language FROM processing WHERE duty LIKE '%VR%'
        UNION ALL
        SELECT '시스템' AS category, language FROM processing WHERE duty LIKE '%시스템%'
        UNION ALL
        SELECT '블록체인' AS category, language FROM processing WHERE duty LIKE '%블록체인%'
        UNION ALL
        SELECT 'ERP' AS category, language FROM processing WHERE duty LIKE '%ERP%'
        UNION ALL
        SELECT '언어별 개발자' AS category, language FROM processing WHERE duty LIKE '%언어별 개발자%'
    ) AS f
    JOIN total_count t ON f.category = t.duty_category
    GROUP BY f.category, 어학조건, t.total_jobs
    ORDER BY f.category, 비율 DESC;
    """

    df = pd.read_sql_query(sql_query, conn)
    df.columns = df.columns.str.strip()

    # ✅ 직무별 테이블 시각화 및 이미지 저장
    for duty in df["직무"].unique():
        df_duty = df[df["직무"] == duty][["어학조건", "비율"]]

        # ✅ 테이블 시각화
        fig, ax = plt.subplots(figsize=(5, 2))
        ax.axis("tight")
        ax.axis("off")

        table = ax.table(
            cellText=df_duty.values, 
            colLabels=df_duty.columns, 
            cellLoc="center", 
            loc="center",
            colColours=["#007bff"] * len(df_duty.columns)
        )

        table.auto_set_font_size(False)
        table.set_fontsize(14)
        table.scale(0.5, 2.5)

        # ✅ 헤더 글씨 색상 및 정렬 조정
        for i, key in enumerate(df_duty.columns):
            cell = table[0, i]
            cell.set_text_props(color="white", ha="center", va="center")

        plt.title(f"{duty} 직무별 어학 요구 사항", fontsize=19, pad=20)  # 제목 크기 조정

        # ✅ 이미지 저장
        img_path = os.path.join(output_dir, f"{duty}.png")
        plt.savefig(img_path, bbox_inches="tight")
        plt.close()  # ❗ show() 없이 close() 수행

        # ✅ 기존 language 컬럼 값 조회 및 업데이트
        cursor.execute("SELECT language FROM duty_analysis WHERE duty = ?;", (duty,))
        existing_language_data = cursor.fetchone()

        if existing_language_data and existing_language_data[0]:
            try:
                language_list = ast.literal_eval(existing_language_data[0])
                if isinstance(language_list, list):
                    language_list.append(img_path)
                else:
                    language_list = [img_path]
            except:
                language_list = [img_path]
        else:
            language_list = [img_path]

        updated_text = str(language_list)
        cursor.execute("UPDATE duty_analysis SET language = ? WHERE duty = ?;", (updated_text, duty))
        conn.commit()

        print(f"{duty} 직무 language 컬럼 업데이트 완료: {updated_text}")

except Exception as e:
    print(f"에러 발생: {e}")

finally:
    conn.close()


AI 직무 language 컬럼 업데이트 완료: ['language_charts/AI.png', '1. AI 직무에서는 특정 언어 요구가 거의 없으며, 공고의 96.66%에서 언어 요구가 드문 편입니다. 2. 영어 능력이 필요한 공고 비율은 3.12%이며, 일본어 능력 요구는 0.22%로 매우 낮습니다. 3. 글로벌 협업이 필요한 경우 일부 언어 능력이 요구될 가능성이 있습니다.', 'Language_charts\\AI.png', 'Language_charts\\AI.png', 'Language_charts\\AI.png']
ERP 직무 language 컬럼 업데이트 완료: ['language_charts/ERP.png', '1. ERP 직무에서는 특정 언어 요구가 거의 없으며, 공고의 94.59%에서 언어 요구가 드문 편입니다. 2. 영어 능력이 필요한 공고 비율은 5.41%이며, 글로벌 프로젝트 및 해외 고객과의 소통을 위해 요구됩니다. 3. ERP 시스템의 글로벌 확산으로 인해 다국적 협업이 필수적입니다.', 'Language_charts\\ERP.png', 'Language_charts\\ERP.png', 'Language_charts\\ERP.png']
PM 직무 language 컬럼 업데이트 완료: ['language_charts/PM.png', '1. PM 직무에서는 특정 언어 요구가 거의 없으며, 공고의 92.1%에서 언어 요구가 드문 편입니다. 2. 영어 능력 요구 비율은 7.24%이며, 일본어는 0.66%로 상대적으로 적습니다. 3. 글로벌 프로젝트에서 원활한 소통을 위해 어학 능력이 요구될 수 있습니다.', 'Language_charts\\PM.png', 'Language_charts\\PM.png', 'Language_charts\\PM.png']
QA 직무 language 컬럼 업데이트 완료: ['language_charts/QA.png', '1. QA 직무에서는 특정 언어 요구가 거의 없으며, 공고의 95.7%에서 언어 요구가 드문 편